## Content

This dataset consists of real-world consumer complaints about financial products and services. Each complaint is labeled with a specific product, framing this as a supervised text classification problem. To classify future complaints based on their content, various machine learning algorithms were employed to improve the accuracy of predictions, assigning each complaint to the appropriate product category.

In [ ]:
# Importing Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
from io import StringIO
import gc